# Supervised ML models: Classification

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
raw_data = pd.read_csv('https://github.com/praisan/hello-world/raw/master/churn_data.csv', delimiter=',')
raw_data.head()

In [ ]:
x = raw_data.drop(['AccountLength',	'State',	'AreaCode',	'Phone',	'Churn'], axis=1)
y=np.ravel(raw_data['Churn'])

In [ ]:
# Train test split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state=50)

# Model training and testing

In [ ]:
# เลือกโมเดลและวิธีการฝึก
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(penalty='l2', tol=0.001, solver='lbfgs', max_iter=500)

In [ ]:
# ฝึกโมเดลด้วยข้อมูลชุดฝึก
lr_model=lr_model.fit(x_train, y_train)

In [ ]:
# ทดสอบโมเดลด้วยข้อมูลชุดทดสอบ
y_predict = lr_model.predict(x_test)

In [ ]:
# วัดประสิทธิภาพ
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_predict))

cm = np.array(confusion_matrix(y_test, y_predict, labels=[0,1]))
confusion = pd.DataFrame(cm, index=['Not Churn', 'Churn'], columns=['Predicted Not Churn', 'Predicted Churn'])
print('Confusion matrix')
confusion

In [ ]:
# Plot ROC
from sklearn import metrics
pred = lr_model.predict_proba(x_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred[:,1])
roc_auc = metrics.auc(fpr, tpr)


def plot_roc(fpr, tpr,roc_auc):
  plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC (AUC = %0.2f)' % (roc_auc))
  plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
          label='Chance', alpha=.8)
  plt.legend(loc="lower right")
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver operating characteristic example')
 # return plt

plot_roc(fpr, tpr,roc_auc)

## --> Hands on 1: ฝึกปฏิบัติทดลองสร้างแบบจำลองทำนายโรคเบาหวานใน LabDay2_Classification_hands_on.ipynb

# Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
zscore_model= StandardScaler().fit(x_train)
lr_scaled = LogisticRegression(penalty='l2', tol=0.001)
lr_scaled=lr_scaled.fit(zscore_model.transform(x_train), y_train)
y_predict_scaled = lr_scaled.predict(zscore_model.transform(x_test))

print(classification_report(y_test, y_predict_scaled))

cm_scaled = np.array(confusion_matrix(y_test, y_predict_scaled, labels=[0,1]))
confusion = pd.DataFrame(cm_scaled, index=['Not Churn', 'Churn'], columns=['Predicted Not Churn', 'Predicted Churn'])
print('Confusion matrix')
confusion


In [ ]:
pred = lr_scaled.predict_proba(zscore_model.transform(x_test))
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred[:,1])
roc_auc = metrics.auc(fpr, tpr)

plot_roc(fpr, tpr,roc_auc)

## --> Hands on 2: ฝึกปฏิบัติทดลองสร้างแบบจำลองทำนายโรคเบาหวานใน LabDay2_Classification_hands_on.ipynb

# บันทึก Model เก็บไว้ใช้งาน

In [ ]:
# ทดลองบันทึก Model ของการจัดกลุ่ม
from joblib import dump, load
dump(zscore_model,'zscore_churn.model')
dump(lr_scaled,'predict_churn.model')


In [ ]:
#ทดลองโหลดมาใช้

zscore_churn = load('zscore_churn.model') 
predict_churn = load('predict_churn.model') 

y_predict_new_data = lr_scaled.predict(zscore_churn.transform(x_test))

print(classification_report(y_test, y_predict_new_data))

cm_scaled = np.array(confusion_matrix(y_test, y_predict_new_data, labels=[0,1]))
confusion = pd.DataFrame(cm_scaled, index=['Not Churn', 'Churn'], columns=['Predicted Not Churn', 'Predicted Churn'])
print('Confusion matrix')
confusion

# Pipeline
Pipeline เป็นโมเดลที่ประกอบด้วยหลายโมเดลที่นำมาวางต่อกันเป็น step โดยจะทำงานตามลำดับของโมเดลที่อยู่ใน step ตั้งแต่ต้นจนจบ

In [ ]:
# ตัวอย่าง classifier ที่มีข้นตอนของการทำ standardization แล้วใช้ logistic regression 
from sklearn.pipeline import Pipeline
models = Pipeline([
  ('zscore', StandardScaler()),
  ('lr_model', LogisticRegression())
])

# ฝึกโมเดลใน Pipeline ด้วยข้อมูลชุดฝึกสอน
models.fit(x_train, y_train)


In [ ]:
# นำ Pipeline มาทำนายข้อมูล
y_predict_new_data = models.predict(x_test)

print(classification_report(y_test, y_predict_new_data))

cm_scaled = np.array(confusion_matrix(y_test, y_predict_new_data, labels=[0,1]))
confusion = pd.DataFrame(cm_scaled, index=['Not Churn', 'Churn'], columns=['Predicted Not Churn', 'Predicted Churn'])
print('Confusion matrix')
confusion

In [ ]:
#ตัวอย่างการเรียกใช้เฉพาะส่วน โดยทดลองเรียก logistic model ที่อยู่ใน Pipeline step 1 (step เริ่มที่ 0, 1, 2, ...)
print(models.steps[1][0])
models.steps[1][1].coef_

In [ ]:
# บันทึกไว้ใช้งานทั้ง Pipeline
dump(models,'predict_churn_pipeline.model')

In [ ]:
#โหลด Pipeline มาใช้งาน
predict_churn_pipeline=load('predict_churn_pipeline.model')
y_predict_new_data = predict_churn_pipeline.predict(x_test)

print(classification_report(y_test, y_predict_new_data))

cm_scaled = np.array(confusion_matrix(y_test, y_predict_new_data, labels=[0,1]))
confusion = pd.DataFrame(cm_scaled, index=['Not Churn', 'Churn'], columns=['Predicted Not Churn', 'Predicted Churn'])
print('Confusion matrix')
confusion

In [ ]:
#ทดลองเรียกใช้โมเดลต่าง ๆ ที่อยู่ใน Pipeline
zscore_model=predict_churn_pipeline.steps[0][1]
logistic_model=predict_churn_pipeline.steps[1][1]
y_predict_scaled = logistic_model.predict(zscore_model.transform(x_test))

print(classification_report(y_test, y_predict_scaled))

cm_scaled = np.array(confusion_matrix(y_test, y_predict_scaled, labels=[0,1]))
confusion = pd.DataFrame(cm_scaled, index=['Not Churn', 'Churn'], columns=['Predicted Not Churn', 'Predicted Churn'])
print('Confusion matrix')
confusion


## --> Hands on 3: ฝึกปฏิบัติทดลองสร้างแบบจำลองทำนายโรคเบาหวานใน LabDay2_Classification_hands_on.ipynb

---
# ใช้ Model อื่นที่สามารถนำมาใช้ในการแยกประเภทได้


* [Decision Tree](https://scikit-learn.org/stable/modules/tree.html#classification) 
 * ตัวอย่างการใช้
```
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
```
 * ตัวอย่างการใช้กับ Pipeline
```
from sklearn.tree import DecisionTreeClassifier
models = Pipeline([
  ('zscore', StandardScaler()),
  ('boot', DecisionTreeClassifier())
```

* [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html#classification)

 * ตัวอย่างการใช้
```
from sklearn.svm import SVC
model = SVC(kernel='rbf')
```
 * ตัวอย่างการใช้กับ Pipeline
```
from sklearn.svm import SVC
models = Pipeline([
  ('zscore', StandardScaler()),
  ('c_model', SVC(kernel='rbf'))
```
* [Neural Network](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#classification)
 * ตัวอย่างการใช้
```
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(5, 2), random_state=1)
```
 * ตัวอย่างการใช้กับ Pipeline
```
from sklearn.neural_network import MLPClassifier
models = Pipeline([
  ('zscore', StandardScaler()),
  ('c_model', MLPClassifier(hidden_layer_sizes=(5, 2), random_state=1))
```






#### Parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

prediction_model = GridSearchCV(
        DecisionTreeClassifier(class_weight={0:1,1:2}), 
        param_grid= [{'criterion': ['entropy','gini']}], 
        scoring='f1_micro',
        cv=5)